# Webscraping | Day03 | API wrappers


In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [2]:
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [3]:
#embedding our getpass credentials in the access key 
sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret))

In [33]:
playlist=sp.user_playlist_tracks('Leckleon','2WrqbQt0mpUlyEnxQBmVfs')

In [34]:
playlist["items"][3]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [35]:
len(playlist["items"])

100

In [36]:
#function which handles more than 100 results 

def get_playlist_tracks(user_id,playlist_id):
    results=sp.user_playlist_tracks(user_id,playlist_id)
    tracks=results["items"]
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks 


In [65]:
results=get_playlist_tracks('nomditecalicious','0SPwoPg9Cew24vbytFLRCo')

In [66]:
len(results)

10

In [67]:
#results = [i for i in results if i]


In [69]:
names=[]
ids=[]
artist=[]
playlist=results

for song in range(len(results)):
    names.append(results[song]['track']['name'])
    ids.append(results[song]['track']['id'])
    artist.append(results[song]['track']['artists'][0]['name'])

In [72]:
playlist_song_info=pd.DataFrame({"names":names,"ids":ids,"artist":artist})

In [73]:
playlist_song_info

,names,ids,artist
0,Funky Kingston,0bJ4nQojefjNpeuZNkiLrX,Toots & The Maytals
1,Hand Clapping Song,3COOd5xqfP3VUeTZtUKcvD,The Meters
2,Summer Days,2YCcYjoyShxR1fko25WTdc,Radio Citizen
3,The Revolution Will Not Be Televised,7kjg2NCn3Zx70m1DFHSSGO,Gil Scott-Heron
4,Whitey on the Moon,6Jnsl9pPYkX8jx0XyzzT84,Gil Scott-Heron
5,Masters Of War,6X30GXDokF0rjLKP0gGEvV,The Staple Singers
6,Funkier Than a Mosquito's Tweeter,5wBi26umb4oxXLmpJ32BWV,Nina Simone
7,Can You Get to That,2liDEMyPVzDDLXKUnyiznA,Funkadelic
8,Love Me,3VgQTiiOCA1vAajq8XHWdh,Marbert Rocel
9,The Dream,3mpREieq9LagvI9PX9Jljw,TrinityRoots


In [74]:
playlist_song_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   names   10 non-null     object
 1   ids     10 non-null     object
 2   artist  10 non-null     object
dtypes: object(3)
memory usage: 368.0+ bytes


In [75]:
song_ID

[]

In [79]:
feats=sp.audio_features(tracks=ids[0:99])

In [80]:
feats[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [84]:
audio_feat_df=pd.DataFrame(sp.audio_features(tracks=ids[0:99]))

In [87]:
from tqdm.notebook import tqdm

In [88]:
audio_features=[]

for ids in tqdm(range(len(playlist_song_info))):
    audio_features.extend(sp.audio_features(tracks=playlist_song_info['ids'][ids]))


  0%|          | 0/10 [00:00<?, ?it/s]

In [89]:
audio_feat_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.768,0.500,1,-9.455,0,0.0554,0.239,0.002020,0.0344,0.960,99.780,audio_features,0bJ4nQojefjNpeuZNkiLrX,spotify:track:0bJ4nQojefjNpeuZNkiLrX,https://api.spotify.com/v1/tracks/0bJ4nQojefjN...,https://api.spotify.com/v1/audio-analysis/0bJ4...,290200,4
1,0.805,0.518,9,-12.802,1,0.0804,0.487,0.853000,0.1020,0.631,90.520,audio_features,3COOd5xqfP3VUeTZtUKcvD,spotify:track:3COOd5xqfP3VUeTZtUKcvD,https://api.spotify.com/v1/tracks/3COOd5xqfP3V...,https://api.spotify.com/v1/audio-analysis/3COO...,176627,4
2,0.836,0.401,9,-10.949,0,0.1480,0.489,0.001260,0.2370,0.477,140.008,audio_features,2YCcYjoyShxR1fko25WTdc,spotify:track:2YCcYjoyShxR1fko25WTdc,https://api.spotify.com/v1/tracks/2YCcYjoyShxR...,https://api.spotify.com/v1/audio-analysis/2YCc...,276453,4
3,0.558,0.677,11,-12.380,0,0.2070,0.535,0.000000,0.1010,0.960,180.685,audio_features,7kjg2NCn3Zx70m1DFHSSGO,spotify:track:7kjg2NCn3Zx70m1DFHSSGO,https://api.spotify.com/v1/tracks/7kjg2NCn3Zx7...,https://api.spotify.com/v1/audio-analysis/7kjg...,185879,4
4,0.644,0.555,1,-11.496,1,0.3300,0.831,0.000004,0.7250,0.691,104.716,audio_features,6Jnsl9pPYkX8jx0XyzzT84,spotify:track:6Jnsl9pPYkX8jx0XyzzT84,https://api.spotify.com/v1/tracks/6Jnsl9pPYkX8...,https://api.spotify.com/v1/audio-analysis/6Jns...,119531,4
5,0.392,0.372,8,-16.575,1,0.0361,0.881,0.064800,0.1050,0.317,131.673,audio_features,6X30GXDokF0rjLKP0gGEvV,spotify:track:6X30GXDokF0rjLKP0gGEvV,https://api.spotify.com/v1/tracks/6X30GXDokF0r...,https://api.spotify.com/v1/audio-analysis/6X30...,275720,3
6,0.530,0.545,8,-15.729,0,0.0535,0.403,0.007690,0.6990,0.783,138.678,audio_features,5wBi26umb4oxXLmpJ32BWV,spotify:track:5wBi26umb4oxXLmpJ32BWV,https://api.spotify.com/v1/tracks/5wBi26umb4ox...,https://api.spotify.com/v1/audio-analysis/5wBi...,320427,4
7,0.689,0.830,8,-8.774,1,0.0954,0.539,0.000000,0.0882,0.815,140.732,audio_features,2liDEMyPVzDDLXKUnyiznA,spotify:track:2liDEMyPVzDDLXKUnyiznA,https://api.spotify.com/v1/tracks/2liDEMyPVzDD...,https://api.spotify.com/v1/audio-analysis/2liD...,171560,4
8,0.749,0.327,3,-11.141,1,0.0313,0.937,0.204000,0.1100,0.394,114.844,audio_features,3VgQTiiOCA1vAajq8XHWdh,spotify:track:3VgQTiiOCA1vAajq8XHWdh,https://api.spotify.com/v1/tracks/3VgQTiiOCA1v...,https://api.spotify.com/v1/audio-analysis/3VgQ...,326973,4
9,0.680,0.439,4,-8.741,0,0.3190,0.044,0.513000,0.1030,0.532,168.946,audio_features,3mpREieq9LagvI9PX9Jljw,spotify:track:3mpREieq9LagvI9PX9Jljw,https://api.spotify.com/v1/tracks/3mpREieq9Lag...,https://api.spotify.com/v1/audio-analysis/3mpR...,538333,4


In [ ]:
#df.to_csv